<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

# Bielik + Ollama – czyli proste uruchomienie LLM

LLM można używać bezpośrednio korzystając z biblioteki **transformers** od HuggigFace, lecz na dłuższą metę może okazać się to uciążliwe – np. trzymanie modelu w obiektach i zarządzanie nimi (często powiązane z obsługą GPU).

Z pomocą przychodzą różne narzędzia takie jak: **Ollama**, **vLLM**, **Text Generation Interface (TGI)** czy **llama.cpp** – w tym tutorialu, ze względu na prostotę, zostanie omówione pierwsze rozwiązanie.

## Ollama

Ollama to narzędzie do uruchamiania i zarządzania dużymi modelami językowymi na lokalnym komputerze. Zapewnia prosty interfejs do korzystania z gotowych modeli, jak również możliwość ich personalizacji i dostosowywania za pomocą specjalnych plików konfiguracyjnych. Ollama jest dostępna na różnych platformach, w tym macOS, Windows oraz Linux.

Przydatne linki:
  - Oficjalna strona projektu: https://ollama.com
  - Github: https://github.com/ollama/ollama
  - Dokumentacja API: https://github.com/ollama/ollama/blob/main/docs/api.md

### Ollama instalacja

Narzędzie jest wspierane na platformach macOS, Windows oraz Linux. Należy zainstalować je jako klasyczną aplikację. Na platformach Windows i macOS będzie to możliwe poprzez klasyczne pobranie i zainstalowanie pakietu, natomiast na Linuxie będzie trzeba uruchomić specjalną komendę *[stan na sierpień 2024]*.

Szczegółowe informacje znajdują się na stronie projektu.

Aplikacja działa w oparciu o Command Line Interface (CLI) – wiersz poleceń. W celu weryfikacji poprawnej instalacji zaleca się  uruchomienie komendy: `ollama`. Przykładowy wynik jej działania został przedstawiony poniżej (a dokładnie instrukcja zawierająca komendy do obsługi tego narzędzia). Jeżeli pojawią się błędy, może oznaczać to, że narzędzie nie zostało zainstalowane poprawnie.

```shell
(base) sample_user@MacBook-Pro SPEAKLEASH % ollama
Large language model runner

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.
```

### Ollama uruchomienie poprzez CLI
W zaprezentowanym przykładzie wykorzystamy model Bielik-11B-v2.2-Instruct-GGUF formacie kwantyzacji Q4_K_M:<br>
```bielik-11b-v2.2-instruct:Q4_K_M``` <br>

Więcej informacji na temat modelu można znaleźć pod linkiem:<br>
https://huggingface.co/speakleash/Bielik-11B-v2.2-Instruct-GGUF<br>

Natomiast informacje dotyczące pozostałych formatów kwantyzacji, które są dostępne w serwisie Ollama, można sprawdzić na stronie:<br>
https://ollama.com/SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M

Aby skorzystać z modelu należy go uprzednio pobrać za pomocą komendy:
```shell
ollama pull SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M
```

Następnie model można uruchomić stosując polecenie:

```shell
ollama run SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M
```

Docelowo można pominąć pierwszy krok, czyli pobieranie, ponieważ uruchomienie modelu za pomocą komedny ```run``` automatycznie sprawdzi, czy jest on dostępny w lokalnej bazie. Jeżeli tak nie jest, to rozpocznie się jego pobieranie.<br>
Po poprawnym załadowaniu będzie można zacząć korzystać z modelu w konsoli.

```shell
(base) marcinwatroba@DataScienceServer:~$ ollama run SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M
>>> Ile to jest 2+4?
<s>  2 plus 4 równa się 6.

>>> Kto napisał Pana Tadeusza?
<s>  Adam Mickiewicz napisał "Pana Tadeusza".

>>>
```

### Ollama – korzystanie z API (curl)

Ollama, poza korzystaniem z CLI, udostępnia również możliwość komunikacji poprzez Rest API.
Pozwala to na korzystanie z modelu w praktycznie każdej aplikacji, co daje całkiem spore możliwości.

Domyślnie aplikacja działa na porcie 11434. Oznacza to, że jej domyślny host to http://localhost:11434 i z niego będziemy korzystać w poniższych przykładach.

Najprostszym przykładem będzie po prostu uruchomienie klasycznego zapytania dla zapytania składającego się z listy wiadomości (szczegóły dotyczące API znajdują się pod adresem https://github.com/ollama/ollama/blob/main/docs/api.md).

Poniżej znajduje się przykład zapytania w popularnej bibliotece `curl` -- można jej używać w formie CLI, bez większych umiejętności programowania.

```shell
curl http://localhost:11434/api/chat -d '{
  "model": "SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_MF",
  "stream": false,
  "messages": [
    {
      "role": "system",
      "content": "Odpowiedz krótko na pytanie"
    },
    {
      "role": "user",
      "content": "Kim jest Adam Mickiewicz?"
    }
  ]
}'
```

Zapytanie ma body w formie JSON, gdzie istotne są 3 parametry: `model`, `stream` i `messages`.
W odpowiedzi otrzymany następująxcy obiekt:
```
{
   "model":"SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M",
   "created_at":"2024-08-21T15:45:22.580717683Z",
   "message":{
      "role":"assistant",
      "content":"\u003cs\u003e  Adam Mickiewicz to polski poeta, uważany za jednego z najważniejszych twórców literatury romantyzmu. Znany m.in. z dzieł \"Pan Tadeusz\" i \"Dziady\"."
   },
   "done_reason":"stop",
   "done":true,
   "total_duration":967596312,
   "load_duration":4041118,
   "prompt_eval_count":44,
   "prompt_eval_duration":34273000,
   "eval_count":65,
   "eval_duration":798981000
}
```

### Ollama – korzystanie z API (python)

Analogiczny przykład z powyższego zapytania można wykonać w środowisku python.
W tym celu należy zainstalować wymagane zależności oraz napisać kod jaki będzie wykonywał zapytanie.




---


**Korzystanie z Ollama w środowisku Google Colab**

Przedstawione poniżej kroki dotyczą uruchomienia Ollamy oraz pobrania modelu w środowisku Google Colab. Podaną sekcję można pominąć w przypadkiu lokalnej egzekucji kodu (kroki do niej zostały przedstawione w początkowej sekcji dokumentu).

Pobranie Ollamy:

In [40]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13344    0 13344    0     0  45967      0 --:--:-- --:--:-- --:--:-- 45855
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Uruchomienie Ollamy, żeby działa w tle:

In [ ]:
!ollama serve > rocama.log 2>&1 &

Pobranie modelu (plik GGUF w formacie kwantyzacji Q4_K_M).
```| tail -n 3``` zostało dodane, aby nie wyświetlać nadmiarowych informacji dotyczących pobierania modelu:

In [41]:
!ollama pull SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M | tail -n 3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 32354c00869f... 100% ▕▏ 6.7 GB                         
pulling f7426507909a... 100% ▕▏  263 B                         
pulling 3685c9d39c8b... 100% ▕▏  114 B                         
pulling dda2439e812c... 100% ▕▏  414 B                         
verifying sha256 digest 
writing manifest 
success 


Do celów testowych zastosowano nazwę domeny 'localhost'. Można również użyć adresu IP wskazujący na serwer Ollamy.



---


Poniżej znajdują się komendy instalujące biblioteki `httpx` i `requests`. Są one alternatywne i dają takie same funkcjonalności -- jeśli nie znasz żadnej wybierz `httpx`.

In [42]:
!pip install -qq httpx requests

In [43]:
# pprint jest dodatkowe i można użyć zwykłego print, który nie formatuje składni
from pprint import pprint

# Stałe dla zapytania — będą tak samo wykorzystane w requests i httpx
URL = "http://localhost:11434/api/chat"
PAYLOAD = {
    "model": "SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M",
    "stream": False,
    "messages": [
        {
            "role": "system",
            "content": "Odpowiedz krótko na pytanie"
        },
        {
            "role": "user",
            "content": "Kim jest Adam Mickiewicz?"
        }
    ]
}

In [45]:
import httpx

response = httpx.post(URL, json=PAYLOAD, timeout=500)
pprint(response.json())

{'created_at': '2024-08-29T22:46:46.311720254Z',
 'done': True,
 'done_reason': 'stop',
 'eval_count': 178,
 'eval_duration': 246262319000,
 'load_duration': 51997919094,
 'message': {'content': 'Adam Mickiewicz był polskim poetą, dramaturgiem i '
                        'publicystą, uznawanym za jednego z najważniejszych '
                        'twórców romantyzmu w Polsce. Urodził się 24 grudnia '
                        '1798 roku w Zaosiu koło Nowogródka na Litwie i zmarł '
                        '26 listopada 1855 roku w Konstantynopolu (obecnie '
                        'Stambuł). Jest autorem takich dzieł jak "Pan '
                        'Tadeusz", "Dziady" czy "Sonety krymskie". Jego '
                        'twórczość miała ogromny wpływ na polską literaturę i '
                        'tożsamość narodową.',
             'role': 'assistant'},
 'model': 'SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M',
 'prompt_eval_count': 103,
 'prompt_eval_duration': 117823341000,
 'total_

In [46]:
import requests

response = requests.post(URL, json=PAYLOAD)
pprint(response.json())

{'created_at': '2024-08-29T22:51:06.824372633Z',
 'done': True,
 'done_reason': 'stop',
 'eval_count': 182,
 'eval_duration': 259194295000,
 'load_duration': 12672613,
 'message': {'content': 'Adam Mickiewicz był polskim poetą, dramaturgiem i '
                        'publicystą, uważanym za jednego z najwybitniejszych '
                        'twórców romantyzmu w literaturze polskiej. Urodził '
                        'się 24 grudnia 1798 roku w Zaosiu koło Nowogródka na '
                        'Litwie, a zmarł 26 listopada 1855 roku w '
                        'Konstantynopolu (obecnie Stambuł). Jest autorem '
                        'takich dzieł jak "Pan Tadeusz", "Dziady" czy "Sonety '
                        'krymskie". Jego twórczość miała ogromny wpływ na '
                        'rozwój polskiej literatury i tożsamości narodowej.',
             'role': 'assistant'},
 'model': 'SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M',
 'prompt_eval_count': 103,
 'prompt_eval_duration'

## Podsumowanie

W tym tutorialu przedstawiono, jak w prosty sposób uruchomić LLM na swoim komputerze. Teraz możesz całkowicie za darmo uruchomić prywatnie LLM i dowolnie z niego korzystać. W przypadku trudniejszych zadań odsyłamy do dokumentacji, gdzie opisane są bardziej zaawansowane funkcjonalności, które z pewnością pokryją bardziej złożone przypadki.